In [1]:
import pandas as pd 
import numpy as np
#ratio = 2:1

In [2]:
itemcf = pd.read_csv("../datasets/itemcf.csv")
itemcf.head()

,session_id,item_id,rank
0,26,3260,1
1,26,5383,2
2,26,16312,3
3,26,27921,4
4,26,2213,5


In [3]:
tfidf = pd.read_csv("../datasets/tfidf.csv")
tfidf.head()

,session_id,item_id,rank
0,26,19185,1
1,26,3260,2
2,26,17740,3
3,26,22166,4
4,26,18418,5


In [6]:
def ensable_df(df1,df2,ratio=0.5):
    '''
    given two df of rank return weighted rank
    ratio = df1_ration:df2_ration = 1:0.5 (default)
    '''
    df1_r,df2_r = 1,1*ratio
    df1["rank"] ,df2["rank"]  = 1/df1["rank"]*df1_r,1/df2["rank"]*df2_r
    re_lst =[]
    for idx in range(100,len(df1)+1,100):
        buf_lst=[]
        df1_s = df1.iloc[idx-100:idx]
        df2_s = df2.iloc[idx-100:idx]
        s_id = df1_s.session_id.unique()
        total_df = (pd.concat([df1_s,df2_s],ignore_index=True))
        for item_id in total_df.item_id.unique():
            if item_id != -1:
                buf_lst.append((s_id[0],item_id,sum(total_df["rank"][total_df["item_id"]==item_id])))
        buf_lst.sort(key=lambda x: x[2],reverse=True)
        re_lst+=(buf_lst[:100])#take top 100
    df = pd.DataFrame(re_lst,columns=['session_id', 'item_id', 'rank'])
    return df

In [7]:
ens_df = ensable_df(itemcf.copy(),tfidf.copy())
ens_df["rank"] = list(range(1,101))*50000
ens_df.head()

,session_id,item_id,rank
0,26,3260,1
1,26,5383,2
2,26,19185,3
3,26,16312,4
4,26,27921,5


In [ ]:
# def multi_proc(df1_path,df2_path,n_jobs):
#     files = list(os.listdir(session_path))
# 	#split = int(len(files)/n_jobs)
#     count=0
#     jobs =[]
#     manager = mp.Manager()#define shared class
#     return_dict = manager.list()
#     while(count != len(files)):
#         for i in range(n_jobs):
#             if count == len(files):
#                 break
#             p = mp.Process(target=sessions2vec,args=(item_path,session_path+files[count],return_dict))
#             count+=1
#             jobs.append(p)
#             p.start()
#         for proc in jobs:
#             proc.join()
#     print(len(return_dict))
#     return return_dict.copy()